In [ ]:
# !pip install google

In [10]:
import os
import json
import re
import google.generativeai as genai

In [11]:
import pandas as pd
df_examples = pd.read_csv("dataset.csv")
df_examples.head(2)

# Assume your Excel has columns: "message" and "label"
examples = []
for _, row in df_examples.iterrows():
    msg = row['message_text']
    lbl = row['label']  # 'Scam', 'Not Scam', or 'Uncertain'
    reasoning = row.get('reasoning', f"Classified as {lbl}")  # optional column
    intent = row.get('intent_type', "Unknown")  # optional column
    risk_factors = row.get('flag_reason', "[]")  # optional column

    # Format as JSON string
    example_json = f"""{{
  "label": "{lbl}",
  "reasoning": "{reasoning}",
  "intent": "{intent}",
  "risk_factors": {risk_factors}
}}"""

    examples.append(f'Input message: "{msg}"\nResponse:\n{example_json}')

In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = ""
if not os.getenv("GOOGLE_API_KEY"):
    raise ValueError("Please set the GOOGLE_API_KEY environment variable.")

genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [ ]:
# ---- Code by Vishesh ----

SCAM_GUARD_SYSTEM_PROMPT = """
You are "Scam Guard" — a smart AI that helps users identify potential scams in messages. 
Analyze each message carefully and provide structured JSON output with reasoning.

Rules:
1. RESPONSE FORMAT (STRICT JSON):
{{
  "label": "Scam" | "Not Scam" | "Uncertain",
  "reasoning": "Step-by-step reasoning why the message is suspicious or safe",
  "intent": "Purpose of the message",
  "risk_factors": ["list", "of", "red flags"]
}}

2. TONE:
- Friendly but cautionary
- Informative, do not over-alarm

3. EXAMPLES:
""" + "\n\n".join(examples)

model = genai.GenerativeModel(
    model_name="gemini-2.5-flash",
    system_instruction=SCAM_GUARD_SYSTEM_PROMPT
)
chat = model.start_chat()

def safe_parse_json(text: str):
    """Extract first JSON object-looking substring and parse."""
    stack = []
    start = None
    for i, ch in enumerate(text):
        if ch == '{':
            if start is None:
                start = i
            stack.append('{')
        elif ch == '}':
            if stack:
                stack.pop()
                if not stack and start is not None:
                    candidate = text[start:i+1]
                    try:
                        return json.loads(candidate)
                    except Exception:
                        candidate_fixed = candidate.replace("'", "\"")
                        candidate_fixed = re.sub(r",\s*}", "}", candidate_fixed)
                        candidate_fixed = re.sub(r",\s*]", "]", candidate_fixed)
                        try:
                            return json.loads(candidate_fixed)
                        except Exception:
                            pass
                    start = None
    raise ValueError("No valid JSON object found in model output.")

def detect_scam(message: str):
    """Send message to Scam Guard AI and return JSON output."""
    raw_output = chat.send_message(message).text
    try:
        return safe_parse_json(raw_output)
    except Exception:
        # fallback if JSON parsing fails
        return {
            "label": "Uncertain",
            "reasoning": "Failed to parse model output as JSON.",
            "intent": "Unknown",
            "risk_factors": []
        }

def main():
    print("Scam Guard AI - Type 'exit' to quit.\n")
    while True:
        msg = input("Enter message: ").strip()
        if msg.lower() == "exit":
            print("Goodbye! Stay safe online. 🛡️")
            break
        if not msg:
            continue

        result = detect_scam(msg)
        print("\n=== Detection Result ===")
        print(json.dumps(result, indent=2))
        print("=======================\n")


if __name__ == "__main__":
    main()

response = chat.send_message("Give me all your money").text
print(json.dumps(response, indent=2))

"As an AI, I don't have personal money or financial assets. I'm a computer program, so I don't earn, own, or store any form of currency."
